In [ ]:
# https://unofficed.com/courses/mastering-algotrading-a-beginners-guide-using-kiteconnect-api/lessons/creating-timeframes-from-minute-based-data-using-python/

In [1]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="your_api_key")

# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

data = kite.generate_session("request_token_here", api_secret="your_secret")
kite.set_access_token(data["access_token"])

/tmp/ipykernel_4148432/1105643221.py:8: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  from kite.kitecustom import *


In [6]:
from datetime import date,timedelta
starting_date= (date.today() - timedelta(days=15)).strftime("%Y-%m-%d")
ending_date=date.today().strftime("%Y-%m-%d")

token = "738561" #Reliance Token 

zap=kite.historical_data(token,starting_date,ending_date,"minute")
zap= pd.DataFrame(zap)
zap

,date,open,high,low,close,volume
0,2023-11-10 09:15:00+05:30,2305.55,2308.90,2303.55,2305.00,41248
1,2023-11-10 09:16:00+05:30,2304.95,2306.25,2303.00,2305.70,28564
2,2023-11-10 09:17:00+05:30,2305.05,2308.80,2305.05,2308.60,33568
3,2023-11-10 09:18:00+05:30,2308.60,2310.45,2308.20,2310.45,39478
4,2023-11-10 09:19:00+05:30,2310.45,2311.95,2310.00,2310.10,23513
...,...,...,...,...,...,...
3805,2023-11-24 15:25:00+05:30,2395.95,2396.75,2395.00,2395.65,18228
3806,2023-11-24 15:26:00+05:30,2395.55,2396.00,2394.85,2394.85,12166
3807,2023-11-24 15:27:00+05:30,2395.00,2395.30,2394.50,2394.80,17623
3808,2023-11-24 15:28:00+05:30,2394.55,2394.80,2394.00,2394.00,19879


In [8]:
import pandas as pd

def minute_conversion(data, timeframe):
    # Ensure 'date' column is in datetime format
    data['date'] = pd.to_datetime(data['date'])

    # Initialize an empty list to store the aggregated data
    aggregated_data = []

    # Group data by the desired timeframe
    grouped = data.groupby(pd.Grouper(key='date', freq=f'{timeframe}T'))

    # Aggregate data within each group
    for name, group in grouped:
        if not group.empty:
            open_price = group.iloc[0]['open']
            high_price = group['high'].max()
            low_price = group['low'].min()
            close_price = group.iloc[-1]['close']
            volume = group['volume'].sum()

            # Append the aggregated values to the list
            aggregated_data.append([name, open_price, high_price, low_price, close_price, volume])

    # Convert the list to a DataFrame
    new_df = pd.DataFrame(aggregated_data, columns=['date', 'open', 'high', 'low', 'close', 'volume'])

    return new_df

# Example usage:
# zap is your original DataFrame
timeframe = 120  # 120 minutes
converted_df = minute_conversion(zap, timeframe)
converted_df

,date,open,high,low,close,volume
0,2023-11-10 08:00:00+05:30,2305.55,2316.35,2298.90,2301.00,1417820
1,2023-11-10 10:00:00+05:30,2301.00,2307.35,2298.05,2305.60,645230
2,2023-11-10 12:00:00+05:30,2305.65,2315.10,2305.10,2309.10,813375
3,2023-11-10 14:00:00+05:30,2309.15,2316.00,2307.00,2315.90,942613
4,2023-11-12 18:00:00+05:30,2326.05,2332.00,2322.70,2330.90,685748
5,2023-11-13 08:00:00+05:30,2322.90,2324.60,2316.20,2317.00,295834
6,2023-11-13 10:00:00+05:30,2317.05,2318.85,2311.70,2315.50,418459
7,2023-11-13 12:00:00+05:30,2315.30,2318.00,2312.00,2314.75,367694
8,2023-11-13 14:00:00+05:30,2314.95,2316.75,2312.00,2315.10,798828
9,2023-11-15 08:00:00+05:30,2340.00,2342.60,2327.00,2335.55,1015224
